<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/FakeNewsXGBoost1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
from google.colab import drive
pd.set_option('display.max_colwidth', -1)

drive._mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Mounted at /content/drive


In [ ]:
df_fake = pd.read_csv('/content/drive/MyDrive/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/gossipcop_real.csv')
df_fake['label']=0
df_real['label']=1
df= df_fake.head(50).append(df_real.head(50))
df.reset_index(drop=True, inplace=True)
n = 50  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
#print(list_df)
overall_df= pd.DataFrame()
for i in range(0,len(list_df)):
  df= list_df[i]
  s  = df['tweet_ids'].str.split('\t').explode().to_frame()
  s= s.fillna(0)
  s.reset_index(drop=True, inplace=True)


  cols = s['tweet_ids'].drop_duplicates(keep='first').tolist()

  df2 = pd.concat([df, pd.crosstab(s.index, s["tweet_ids"],dropna=True)[cols]], axis=1).replace(
      {1: 1, 0: 0}
  )


  df2.dropna(subset = ["id"], inplace=True)
  overall_df= overall_df.append(df2)



df2 = overall_df.fillna(0)
print(df2.shape)

#df_retweets=df['tweet_ids'].str.split('\t',expand=True)


(100, 12703)


In [ ]:
df2.reset_index(drop=True, inplace=True)
n = 50  #chunk row size
list_df = [df2[i:i+n] for i in range(0,df2.shape[0],n)]
overall_df= pd.DataFrame( )

for i in range(0,len(list_df)):
  df2=list_df[i]
  s  = df2['title'].str.split(' ').explode().to_frame()
  s= s.fillna(0)
  s.reset_index(drop=True, inplace=True)

  cols = s['title'].drop_duplicates(keep='first').tolist()

  df3 = pd.concat([df2, pd.crosstab(s.index, s["title"],dropna=True)[cols]], axis=1).replace(
      {1: 1, 0: 0}
  )
 
  df3.dropna(subset = ["id"], inplace=True)

  overall_df= overall_df.append(df3)
  

df3= overall_df.fillna(0)


df3= df3.drop(['tweet_ids', 'news_url','id','title'], axis=1)



ValueError: ignored

In [ ]:
#df3.columns
df3 = df3.loc[:,~df3.columns.duplicated()]
print(df3.columns)

Index([                'id',           'news_url',              'title',
                'tweet_ids',              'label', '284329075902926848',
       '284332744559968256', '284335412590297089', '284359918792880128',
       '284385385151332353',
       ...
                 '"F--king',             'Moron"',              'Quavo',
          '"Misunderstood"',            'Delilah',              'Son's',
                  'Suicide',      'Heartbreaking',            'Goodbye',
                     'Note'],
      dtype='object', length=13122)


In [ ]:


# Create arrays for the features and the target: X, y
X, y = df3.iloc[:, 1:], df3['label']
print(X.columns)

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

Index([          'news_url',              'title',          'tweet_ids',
                    'label', '284329075902926848', '284332744559968256',
       '284335412590297089', '284359918792880128', '284385385151332353',
       '284456388833452032',
       ...
                 '"F--king',             'Moron"',              'Quavo',
          '"Misunderstood"',            'Delilah',              'Son's',
                  'Suicide',      'Heartbreaking',            'Goodbye',
                     'Note'],
      dtype='object', length=13121)


ValueError: ignored